In [ ]:
# This Python code scrapes the details of each fight from each UFC event page on ufcstats.com 

import requests
import lxml.html as lh
import pandas as pd
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup

# scrape all of the links from the page containing the links to all events
url = "http://ufcstats.com/statistics/events/completed?page=all"
html = urlopen(url)
soup = BeautifulSoup(html, 'lxml')
text = soup.get_text()
links = []
for link in soup.findAll('a', attrs={'href': re.compile("http://")}):
    links.append(link.get('href'))
    
#from the links, take only those that are links to events   
events = []
for link in links:
    if 'http://ufcstats.com/event-details/' in link:
        events.append(link)
    
everyufcfight = []

for url in events:
    #Create a handle, page, to handle the contents of the website
    page = requests.get(url)

    #Store the contents of the website under doc
    doc = lh.fromstring(page.content)

    #Parse data that are stored between <tr>..</tr> of HTML
    tr_elements = doc.xpath('//tr')

    # The table of fight stats has 10 columns. We create a list and add only the row elements that are of length 10.
    fights = []
    for t in tr_elements:
        if len(t) == 10:
           fights.append(t)

    #Create empty list to store data
    col=[]
    i=0

    #For each row, store each first element (header) and an empty list
    for t in fights[i]:
        i+=1
        name=t.text_content()
        col.append((name,[]))

    #Since our first row is the header, data is stored from the second row onwards
    for j in range(1,len(fights)):
        #T is our j'th row
        T=fights[j]

        #i is the index of our column
        i=0

        #Iterate through each element of the row
        for t in T.iterchildren():
            data=t.text_content() 
            #Check if row is empty
            if i>0:
            #Convert any numerical value to integers
                try:
                    data=int(data)
                except:
                    pass
            #Append the data to the empty list of the i'th column
            col[i][1].append(data)
            #Increment i for the next column
            i+=1    

    # Create a dictionary from the extracted data
    Dict={title:column for (title,column) in col}

    #Create a pandas dataframe from the dictionary
    df=pd.DataFrame(Dict)

    # Clean the data by removing the "\n" suffix attached to each cell element
    df = df.replace('\n',' ', regex=True)

    # Convert the df of fight stats to a list, remove the spaces from the strings, 
    # then add the list of data for each fight to the list for each card, and the
    # list for each card to the list of total fights
    
    card = df.values.tolist()
    fightdata = []
    for fight in card:
        for data in fight:
            singlefight = str(data)
            singlefight  = singlefight.split()
            fightdata.append(singlefight)
    everyufcfight.append(fightdata)
            
everyufcfight